In [35]:
class Opion():
    
    def __init__(self):
        self.dataroot= './data/RAFDB_test'
        self.coarse_dataroot= './data/coarse_RAFDB_test' 
        self.maskroot= './data/mask'
        self.batchSize= 1   # Need to be set to 1
        self.fineSize=224 # image size
        self.input_nc=3  # input channel size for first stage
        self.input_nc_g=6 # input channel size for second stage
        self.output_nc=3# output channel size
        self.ngf=64 # inner channel
        self.ndf=64# inner channel
        self.which_model_netD='basic' # patch discriminator
        self.which_model_netF='feature'# feature patch discriminator
        self.which_model_netG='unet_assemble'# seconde stage network
        self.triple_weight=1
        self.name='hybrid_grad'
        self.n_layers_D='3' # network depth
        self.gpu_ids=[0]
        self.model='hybrid_net'
        self.checkpoints_dir='checkpoints/' #
        self.norm='instance'
        self.fixed_mask=1
        self.use_dropout=False
        self.init_type='normal'
        self.mask_type='random'
        self.lambda_A=100
        self.threshold=5/16.0
        self.stride=1
        self.shift_sz=1 # size of feature patch
        self.mask_thred=1
        self.bottleneck=512
        self.gp_lambda=10.0
        self.ncritic=5
        self.constrain='MSE'
        self.strength=1
        self.init_gain=0.02
        self.cosis=1
        self.gan_type='lsgan'
        self.gan_weight=0.2
        self.overlap=4
        self.skip=0
        self.display_freq=1000
        self.print_freq=50
        self.save_latest_freq=5000
        self.save_epoch_freq=2
        self.continue_train=False
        self.epoch_count=1
        self.phase='train'
        self.which_epoch=''
        self.niter=20
        self.niter_decay=100
        self.beta1=0.5
        self.lr=0.0002
        self.lr_policy='lambda'
        self.lr_decay_iters=50
        self.isTrain=True

    

In [36]:
import time
from util.data_load import Data_load
from models.models import create_model
import torch
import os
import torchvision
from torch.utils import data
import torchvision.transforms as transforms
opt = Opion()
transform_mask = transforms.Compose(
    [transforms.Resize((256,256)),
     transforms.ToTensor(),
    ])
transform = transforms.Compose(
    [
     transforms.Resize((256,256)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.5] * 3, std=[0.5] * 3)])

dataset_test = Data_load('test',opt.dataroot,opt.coarse_dataroot, opt.maskroot, transform, transform_mask)

iterator_test = (data.DataLoader(dataset_test, batch_size=opt.batchSize,shuffle=False))

model = create_model(opt)
total_steps = 0


hybrid_net
initialize network with normal
initialize network with normal
initialize network with normal
---------- Networks initialized -------------
UnetGeneratorHybrid(
  (model): UnetSkipConnectionBlock_3_outer(
    (model): Sequential(
      (0): Conv2d(12, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): UnetSkipConnectionBlock_3_outer(
        (model): Sequential(
          (0): LeakyReLU(negative_slope=0.2, inplace=True)
          (1): Conv2d(128, 128, kernel_size=(4, 4), stride=(2, 2), padding=(3, 3), dilation=(2, 2))
          (2): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (3): LeakyReLU(negative_slope=0.2, inplace=True)
          (4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (5): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (6): UnetSkipConnectionBlock_3_outer(
            (model): Sequential(
              (0): LeakyReL

In [38]:
load_epoch=30
model.load(load_epoch)

In [39]:
save_dir = "./result/"
if os.path.exists(save_dir) is False:
    os.makedirs(save_dir)

epoch=1
i=0
count=0
for image, coarse_image, mask,img_name in (iterator_test):
    name=''.join(img_name)
    count+=1
    iter_start_time = time.time()
    image=image.cuda()
    coarse_image=coarse_image.cuda()
    mask=mask.cuda()
    mask=mask[0][0]
    mask=torch.unsqueeze(mask,0)
    mask=torch.unsqueeze(mask,1)
    mask=mask.byte()

    model.set_input(image,coarse_image,mask)
    model.set_gt_latent()
    model.test()
    real_A,real_B,coarse,fake_B=model.get_current_visuals()
    
    pic = (fake_B + 1) / 2.0
    torchvision.utils.save_image(pic, save_dir+name)